In [1]:
import os, json
import numpy as np
import pandas as pd
from dLoader import DataLoader

# Get Tickers from Database

In [2]:
directory = os.path.join(os.getcwd(), 'Database')
# Search Param
min_dollar = 0
max_dollar = 10
min_length = 250 * 6
today = pd.Timestamp.now()
# Placeholder
tickers = []
files = os.listdir(directory)
if 'keeper.json' in files:
    path = os.path.join(directory, 'keeper.json')
    db = pd.read_json(path, orient='index')
    # Get pricing
    mask = (db['Latest Price'].between(min_dollar, max_dollar)) & (db['Length in Database'] > min_length)
    ndb = db.loc[mask].copy()
    need_update = (today.day - pd.to_datetime(ndb['Last Updated']).dt.day) > 90
    if need_update.sum() > 0:
        jFile = json.load(open(path, 'r'))
        for ticker in ndb.loc[need_update].index.to_list():
            DataLoader(ticker).update_database()
            jFile[ticker]['Last Updated'] = today.strftime('%Y-%m-%d')
        # Write json file
        json.dump(jFile, open(path, 'w'), indent=4)
    tickers = db.loc[mask].index.to_list()
else:
    jFile = {}
    for file in files:
        name, ext = file.split('.')
        if ext == 'csv':
            df = DataLoader(name).data
            latest_price = df['Close'].iloc[-1]
            jFile[name] = {'Latest Price': latest_price,
                           'Length in Database': len(df),
                           'Last Updated': df.index[-1].strftime('%Y-%m-%d')}
            if len(df) > min_length and (min_dollar < latest_price < max_dollar):
                tickers.append(name)
    # Write json file
    with open(os.path.join(directory, 'keeper.json'), 'w') as outfile:
        json.dump(jFile, outfile, indent=4)

print('Total tickers met parameters in Database: {}'.format(len(tickers)))

Total tickers met parameters in Database: 253
